In [ ]:
# bifurcation search over all the symmetric parameters

import pickle
import numpy as np
import matplotlib.pyplot as plt

from time import time
from numba import jit
from tqdm import tqdm
from scipy.integrate import odeint
from scipy.signal import periodogram
from multiprocessing import Pool

import warnings
warnings.filterwarnings("ignore")
error_message = 'Excess work done on this call (perhaps wrong Dfun type).'

In [ ]:
@jit(nopython=True)
def DvD(S, t, k, K):
    """
    The function of the system for scipy.integrate.odeint.
    
    Parameters
    --------------
    S : array
    Condition of substrates 
    t : array
    A sequence of time points.
    k : array
    Rate constants.
    K: array
    MM constants.
    
    Returns
    ----------
    Sintg : array
    The change of S.
    """
    
    Sintg = np.empty(6)
    Sa_00, Sa_01, Sa_10, Sb_00, Sb_01, Sb_10 = S
    
    E = 20./(1 + Sa_00/K[0] + Sa_00/K[1] + Sa_01/K[2]   + Sa_10/K[3]
                  + Sb_00/K[8] + Sb_00/K[9] + Sb_01/K[10] + Sb_10/K[11])
    F = 20./(1 + Sa_01/K[4]   + Sa_10/K[5]   + (1000.-Sa_00-Sa_01-Sa_10)/K[6]   + (1000.-Sa_00-Sa_01-Sa_10)/K[7]
                  + Sb_01/K[12] + Sb_10/K[13] + (1000.-Sb_00-Sb_01-Sb_10)/K[14] + (1000.-Sb_00-Sb_01-Sb_10)/K[15])
             
    Sintg[0] = - k[0]*E*Sa_00/K[0] - k[1]*E*Sa_00/K[1] + k[4]*F*Sa_01/K[4] + k[5]*F*Sa_10/K[5]
    Sintg[1] = - k[4]*F*Sa_01/K[4] - k[2]*E*Sa_01/K[2] + k[0]*E*Sa_00/K[0] + k[6]*F*(1000.-Sa_00-Sa_01-Sa_10)/K[6]
    Sintg[2] = - k[5]*F*Sa_10/K[5] - k[3]*E*Sa_10/K[3] + k[1]*E*Sa_00/K[1] + k[7]*F*(1000.-Sa_00-Sa_01-Sa_10)/K[7]
    Sintg[3] = - k[8]*E*Sb_00/K[8] - k[9]*E*Sb_00/K[9] + k[12]*F*Sb_01/K[12] + k[13]*F*Sb_10/K[13]
    Sintg[4] = - k[12]*F*Sb_01/K[12] - k[10]*E*Sb_01/K[10] + k[8]*E*Sb_00/K[8] + k[14]*F*(1000.-Sb_00-Sb_01-Sb_10)/K[14]
    Sintg[5] = - k[13]*F*Sb_10/K[13] - k[11]*E*Sb_10/K[11] + k[9]*E*Sb_00/K[9] + k[15]*F*(1000.-Sb_00-Sb_01-Sb_10)/K[15]
             
    return(Sintg)

@jit
def check_convergence(v, trange, epsilon=1.0):
    """
    Judge if each state of a substrate is convergent.
    
    Parameters
    --------------
    v : array
    A sequence of a state of a substrate.
    trange : int
    The time the integration was done.
    epsilon : scalar
    A threshold for the judge.
    
    Returns
    ----------
    1 if not convergence.
    """
    
    rang = trange//10
    
    # check convergence
    vstd = np.std(v[-rang:])
    
    diffstd = np.std(np.diff(v[-rang:]))
    if diffstd < epsilon:
        return(3)
    elif vstd < epsilon:
        return(0)
    else:
        return(1) # not convergence
    
@jit(nopython=True)
def DvD(S, t, k, K):
    """
    The function of the system for scipy.integrate.odeint.
    
    Parameters
    --------------
    S : array
    Condition of substrates 
    t : array
    A sequence of time points.
    k : array
    Rate constants.
    K: array
    MM constants.
    
    Returns
    ----------
    Sintg : array
    The change of S.
    """
    
    Sintg = np.empty(6)
    Sa_00, Sa_01, Sa_10, Sb_00, Sb_01, Sb_10 = S
    
    E = 20./(1 + Sa_00/K[0] + Sa_00/K[1] + Sa_01/K[2]   + Sa_10/K[3]
                  + Sb_00/K[8] + Sb_00/K[9] + Sb_01/K[10] + Sb_10/K[11])
    F = 20./(1 + Sa_01/K[4]   + Sa_10/K[5]   + (1000.-Sa_00-Sa_01-Sa_10)/K[6]   + (1000.-Sa_00-Sa_01-Sa_10)/K[7]
                  + Sb_01/K[12] + Sb_10/K[13] + (1000.-Sb_00-Sb_01-Sb_10)/K[14] + (1000.-Sb_00-Sb_01-Sb_10)/K[15])
             
    Sintg[0] = - k[0]*E*Sa_00/K[0] - k[1]*E*Sa_00/K[1] + k[4]*F*Sa_01/K[4] + k[5]*F*Sa_10/K[5]
    Sintg[1] = - k[4]*F*Sa_01/K[4] - k[2]*E*Sa_01/K[2] + k[0]*E*Sa_00/K[0] + k[6]*F*(1000.-Sa_00-Sa_01-Sa_10)/K[6]
    Sintg[2] = - k[5]*F*Sa_10/K[5] - k[3]*E*Sa_10/K[3] + k[1]*E*Sa_00/K[1] + k[7]*F*(1000.-Sa_00-Sa_01-Sa_10)/K[7]
    Sintg[3] = - k[8]*E*Sb_00/K[8] - k[9]*E*Sb_00/K[9] + k[12]*F*Sb_01/K[12] + k[13]*F*Sb_10/K[13]
    Sintg[4] = - k[12]*F*Sb_01/K[12] - k[10]*E*Sb_01/K[10] + k[8]*E*Sb_00/K[8] + k[14]*F*(1000.-Sb_00-Sb_01-Sb_10)/K[14]
    Sintg[5] = - k[13]*F*Sb_10/K[13] - k[11]*E*Sb_10/K[11] + k[9]*E*Sb_00/K[9] + k[15]*F*(1000.-Sb_00-Sb_01-Sb_10)/K[15]
             
    return(Sintg)

@jit
def check_convergence(v, trange, epsilon=1.0):
    """
    Judge if each state of a substrate is convergent.
    
    Parameters
    --------------
    v : array
    A sequence of a state of a substrate.
    trange : int
    The time the integration was done.
    epsilon : scalar
    A threshold for the judge.
    
    Returns
    ----------
    1 if not convergence.
    """
    
    rang = trange//10
    
    # check convergence
    vstd = np.std(v[-rang:])
    
    diffstd = np.std(np.diff(v[-rang:]))
    if diffstd < epsilon:
        return(3)
    elif vstd < epsilon:
        return(0)
    else:
        return(1) # not convergence

def change_kK_asym(param, idx, val):
    """
    Change the given parameter for the bifurcation analysis.
    Note that the symmetry is broken for the output.
    
    Paramters
    -------------
    param : array, shape (32)
    The parameter on which to do bifurcation analysis.
    idx : scalar
    Specify which constant to change.
    val : float
    The value the target constant is changed to.
    
    Returns
    ----------
    k : array, shape (16)
    Rate constants.
    K : array, shape(16)
    MM constants.
    """
    
    if idx<4: # ka_0-3 (0-3 -> 0-3)
        param[idx] = val
    elif idx<8: # kb_0-3 (4-7 -> 8-11)
        param[idx+4] = val
    elif  idx<12: # Kma_0-3 (8-11 -> 16-19)
        param[idx+8] = val
    elif idx<16: # Kmb_0-3 (12-15 -> 24-28)
        param[idx+12] = val
    k = param[:16]
    K = param[16:]
    return(k, K)

def change_kK_sym(param, idx, val):
    """
    Change the given parameter for the bifurcation analysis.
    Note that the symmetry is maintained for the output.
    
    Paramters
    -------------
    param : array, shape (32)
    The parameter on which to do bifurcation analysis.
    idx : scalar
    Specify which constant to change.
    val : float
    The value the target constant is changed to.
    
    Returns
    ----------
    k : array, shape (16)
    Rate constants.
    K : array, shape(16)
    MM constants.
    """
    
    if idx<4: # ka_0-3 (0-3 -> 0-3)
        param[idx] = val
        param[7-idx] = val
    elif idx<8: # kb_0-3 (4-7 -> 8-11)
        param[idx+4] = val
        param[19-idx] = val
    elif  idx<12: # Kma_0-3 (8-11 -> 16-19)
        param[idx+8] = val
        param[31-idx] = val
    elif idx<16: # Kmb_0-3 (12-15 -> 24-28)
        param[idx+12] = val
        param[43-idx] = val
    k = param[:16]
    K = param[16:]
    return(k, K)

def bifurcation(args):
    """
    Iterate bifurcation analysis.
    Bifurcation result is saved for each constant in each parameter set.
    
    Parameters
    --------------
    args : tuple, shape (4)
        i_core : int
            Specify which cpu core is used.
        idxs : list
            List of indice that specify which constant to analyse.
        params : list
            List of chaotic parameters on which to perform bifurcation analysis.
            Each param is [k, K]. k and K are arrays with shape (16).
            
    Other parameters
    ----------------------
    bif : array, shape (16)
    An array bifurcation results are saved in. 
    The results correspond to [ka1, ..., kb1, ..., Kma1, ..., Kmb1, ...].
    Each value of the result indicates: 0 if convergence, 0.5 if oscillation, 1 if chaos.
    """
    
    i_core, idxs, params = args
    kstep = 0.05
    Kstep = kstep*5/3
    kvals = np.arange(0, 3, kstep)
    Kvals = np.arange(-2, 3, Kstep)
    
    n_step = kvals.shape[0]
    
    st = time()
    S0 = np.asarray([1000., 0., 0., 1000., 0., 0.])
            
    for i_param, param in enumerate(params):
        kp, Kp = param
        p = np.empty(32)
        p[:16] = kp
        p[16:] = Kp
        p = np.log10(p)
        
        bif = np.zeros(n_step)
        judge = np.zeros(6, dtype='int')

        for idx in tqdm(idxs):
            if idx<8:
                vals = kvals
            else:
                vals = Kvals

            for i_val, val in tqdm(enumerate(vals)):
                k, K = change_kK_asym(p, idx, val) # symmetry is broken.
                # k, K = change_kK_sym(p, idx, val) # symmetry is maintained.
                
                trange = 1000
                b = 0
        
                # quick integration
                S, info = odeint(func=DvD, y0=S0, t=np.arange(0, trange, 0.02), args=(k, K), atol=5.0e-4, rtol=5.0e-4, full_output=1)
                if error_message==info['message']:
                    pass
                else:
                    for col in range(6):
                        judge[col] = check_convergence(v=S[:, col], trange=trange*50)
                    if 1 in judge:
                        # integration again
                        S, info = odeint(func=DvD, y0=S0, t=np.arange(0, trange, 0.02), args=(k, K), full_output=1)
                        if error_message==info['message']:
                            pass
                        else:
                            for col in range(6):
                                judge[col] = check_convergence(v=S[:, col], trange=trange*50)
                            if 1 in judge:
                                trange = 6000
                                # careful integration
                                S, info = odeint(func=DvD, y0=S[-1, :], t=np.arange(0, trange, 0.02), args=(k,K), mxstep=10000, atol=1.0e-5, rtol=1.0e-5, full_output=1)
                                if error_message == info['message']:
                                    pass
                                else:
                                    for col in range(6):
                                        judge[col] = check_convergence(v=S[:, col], trange=trange*50)

                                    # if not convergent, judge if oscillatory or chaotic. 
                                    if 1 in judge:
                                        f, Spw = periodogram(S[np.int(trange*50/2):], fs=50, axis=0)
                                        maxfq_row = np.argmax(Spw)//Spw.shape[1]
                                        maxfq_col = np.argmax(Spw)%Spw.shape[1]
                                        maxfq_rate = np.sum(Spw[maxfq_row-2:maxfq_row+3, maxfq_col])/np.sum(Spw[:, maxfq_col])
                                        if 0.15 > maxfq_rate:
                                            b = 1 #chaos 
                                        else:
                                            b = 0.5 # oscillation
                    
                bif[i_val] = b    
                
                savename = './bifurcation{}_param{:03}.pickle'
                #with open(savename.format(idx, i_param), 'wb') as f:
                #    pickle.dump(bif, f)
                
                
def multi_bifurcation(params, n_cores=1):
    args = []
    idxs = np.arange(16)
    args = [(i_core, idxs[i_core::n_cores], params) for i_core in range(n_cores)]
    
    with Pool(processes=n_cores) as pool:
        result = pool.map(bifurcation, args)

In [ ]:
params = [[np.random.rand(16), np.random.rand(16)] for _ in range(2)] # load parameter sets you want to perform bifurcation analysis on.
multi_bifurcation(params, 2)